In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [261]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [338]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread('Makro Invoices/Makro Invoices/Converted/64622_A30013841 TIPO BB8B8_20220503_6_FacturaCompra_page1.png', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b
            vat.append(b)
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
        if (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
        if (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
        if (re.search("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)
            dat["Date"] = b[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
                
            if b[-1]=='€':
                dat["Total"] = b[-2]
            if b[-1] == "|":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
        

<class 'list'>
Wall time: 5.75 s


In [339]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(dat)

{'Vat Number': 'A30013841', 'Total': 0, 'Date': '6420321071', 'Invoice_Number': ''}


In [278]:
bp = dat

In [279]:
data = pd.DataFrame(bp,index=[0])

In [280]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Vat Number,Total,Date,Invoice_Number
0,B76744861,"200,89",04/02/2020,


In [281]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [282]:
df2 = formats.query("last == True")

In [283]:
df2

,formato,nombre,nif,last
9419,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12062,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12178,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
12780,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
16962,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
17485,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18171,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18852,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
18939,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True
20388,\d\d\d\d\d\d\d\d,MAPIESTACION,B76744861,True


In [286]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile('[A-Z]{1}[0-9]{2}/[0-9]{4}')
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [287]:
dat

{'Vat Number': 'B76744861',
 'Total': '200,89',
 'Date': '04/02/2020',
 'Invoice_Number': 'F20/2473'}

In [252]:
text
text_list = text.split("\n")
print(text_list)

['Coca-Cola European Partners Iberia, S.L.U., Una compatia perteneciente al grupo Coca-Cola European Partners, P.L.C. C/ Ribera del Loira 20-22 - 2* Planta - 28042 MADRID, C.LF. B-86561412, Registro Mercantil de Madsid, Tomo 32.531, Folio 215, Seccién 8, Hoja M-S46165', '', 'EUROPEAN PARTNERS', 'Ed. ILUMINA - C/ Gaspar Fabregas, 81 - 2." planta', '08950 Esplugues de Liobregat (Barcelona)', '', 'RAZON SOCIAL', '', 'Atencién al Cliente: 900 246 500', '', 'DIRECCION DE ENVIO', '', '19780715 / ’', '', 'IMAPIESTACION SL', '', 'POL LAS TORRES 79 COOPERATIVA', '', '|38670-ADEJE CASCO (Espafia) |', '', '[36010 BAR REST SAN SEBASTIAN', 'ICRT GRAL C288 KM 114 COOPERATIVA S S/N', "|'38670-ADEJE CASCO (Espafia)", '', 'CIFINIF: B76744861 F', 'DOCUMENTO NOMERO FECHA FORMA DE PAGO ‘GRUPO DE GBBRO FECHAVTO. PAG', '04.02.2020 CONTADO 04.02.2020 12', '‘CLIENTE PUNTO DE VENTA DIRECCION TELEFONO', '36010 BAR REST SAN SEBASTIAN CRT GRAL C288 KM 114 COOPERATIVA S S/N 38670-ADEJE CASCO 677509032', '[Transpor

In [268]:
im = Image.open(r"Makro Invoices/Makro Invoices/New/04.02.20_Coca-Cola European Partners Ib_442,51_page1.png") 
# Size of the image in pixels (size of orginal image) 
# (This is not mandatory) 
width, height = im.size 
print(im.size)
# Setting the points for cropped image 
left = 605
top = 800
right = 1678
bottom = 1580
# Cropped image of above dimension 
# (It will not change orginal image) 
im1 = im.crop((left, top, right, bottom)) 
# Shows the image in image viewer 
im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")

(1157, 1649)


In [269]:
img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img,config='outputbase digits'))

In [270]:
text_list = text.split("\n")
text_list = text_list
print(text_list)

['92.16', '', '23.04', '', '8', '', '52.32', '', '']


In [271]:
text = text.replace(' ', '')
text = text.replace('-', '')
text

'92.16\n\n23.04\n\n8\n\n52.32\n\n'

In [272]:
for i in text_list:
    if len(i)>=5 and len(i)<=8:
        dat["Total"]  = i

In [273]:
dat

{'Vat Number': ['B-86561412'],
 'Total': '52.32',
 'Date': '8587882309',
 'Invoice_Number': '0000080382'}

In [288]:
new = pd.DataFrame(dat,index=[0])
finals = finals.append(new)
finals.head()

,Vat Number,Total,Date,Invoice_Number
0,A38089132,0,04/02/20,ORD-3437-00003710-QL
0,B-86561412,52.32,8587882309,0000080382
0,B-86561412,52.32,8587882309,0000080382
0,B76744861,"200,89",04/02/2020,F20/2473


In [226]:
finals  =  pd.DataFrame(dat,index=[0])
final = final.append(finals)


In [227]:
final

,Unnamed: 0,Vat Number,Total,Date,Invoice_Number
0,0.0,A-38003414,"90,21",30/06/2020,1600359177
1,0.0,A-38003414,"154,05",27-05-2021,1400804213
2,0.0,G-38055612,"49,52",01/07/2020,2020-00116355
3,0.0,B76546787,siguiente,02/08/2021,721000050614
4,0.0,B76744861,"www.gmfood.es,",03/02/2020,200302180004999
5,0.0,B38574281,TOTAL,03/07/2020,F1-1-01353
6,0.0,"B-76,305.333",0,01/11/2020,2020/129
7,0.0,A30013841,0,03/05/2022,1303205930
8,0.0,78548857-Q,FACTURA,01/08/2021,19/2021
9,0.0,G-38055612,"49,52",01/07/2020,2020-00116355
